In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
import matplotlib.pyplot as plt
import cv2
import json
import os
from collections import defaultdict
import numpy as np
from skimage import draw
import pandas as pd
import copy
import time
from tqdm import tqdm

In [2]:
carpart_config = '../checkpoints/carpart/swa_carpart.py'
carpart_checkpoint = '../checkpoints/carpart/swa_carpart.pth'
carpart_model = init_detector(carpart_config, carpart_checkpoint, device='cuda:2')

/mmdetection/mmdet/core/anchor/builder.py:15: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  warnings.warn(


load checkpoint from local path: ../checkpoints/carpart/swa_carpart.pth


In [4]:
classes = carpart_model.CLASSES
classes

['sli_side_turn_light',
 'tyre',
 'alloy_wheel',
 'hli_head_light',
 'hood',
 'fwi_windshield',
 'flp_front_license_plate',
 'door',
 'mirror',
 'handle',
 'qpa_quarter_panel',
 'fender',
 'grille',
 'fbu_front_bumper',
 'rocker_panel',
 'rbu_rear_bumper',
 'pillar',
 'roof',
 'blp_back_license_plate',
 'window',
 'rwi_rear_windshield',
 'tail_gate',
 'tli_tail_light',
 'fbe_fog_light_bezel',
 'fli_fog_light',
 'fuel_tank_door',
 'lli_low_bumper_tail_light']

In [4]:
#convert annotation segmentation to binary mask
def draw_binary_mask(seg, img_shape):
    seg = np.array(seg).reshape(-1,2)
    polygon = np.array(seg)
    mask = draw.polygon2mask(img_shape, polygon)
    return mask

In [5]:
#compute iou between ground truth and predicted binary masks
def iou(ground_truth_mask, predict_mask):
    area1 = np.sum(ground_truth_mask)
    area2 = np.sum(predict_mask)
#     print(area1, area2)
    #intersections
    intersection = np.sum(np.multiply(ground_truth_mask, predict_mask))
#     print('intersection', intersection)
    union = np.sum(np.add(ground_truth_mask,predict_mask)>0) - intersection
#     print('intersection', intersection, 'uni',union)
    iou = intersection/union
    return iou

In [6]:
def define_damage_size(seg, img_shape):
    seg = np.array(seg).reshape(-1,2).astype(np.int32)
    area = cv2.contourArea(seg)
    if area == 0:
        area = 1e-5
    ratio = area/(img_shape[0]*img_shape[1])
    if ratio <= 5.000000000000013e-05: size = 'Small'
    elif ratio <= 0.2: size = 'Medium'
    else:
        ratio = 'Large'
    return size

# Old data

In [8]:
imgs_folder = '../data/coco_datasets/datasets/scratch/images/'
print(len([entry for entry in os.listdir(imgs_folder) if os.path.isfile(os.path.join(imgs_folder, entry))]))


23708


In [9]:
# iou_dict = defaultdict(list)
scratches_on_carparts = defaultdict(int)
small_scratch = defaultdict(int)
medium_scratch = defaultdict(int)
large_scratch = defaultdict(int)

In [10]:
# file_name = 'test_train_valid_combination'
json_path='../data/coco_datasets/datasets/scratch/annotations/test_train_valid_combination.json'
# with open(os.path.join(json_path, file_name+'.json'), 'w') as json_file:
#     json.dump(total_data, json_file, indent=4)
total_data = json.load(open(json_path))

In [11]:
#imgToAnns dict: mapping an image Id to a list of annotations
anns = {}
imgToAnns = defaultdict(list)
for ann in total_data['annotations']:
    anns[ann['id']] = ann
    imgToAnns[ann['image_id']].append(ann)


In [12]:
print('number of annotations: ', len(anns))
print('number of images: ', len(total_data['images']))
# print(imgId_to_img[2021].shape)

number of annotations:  120119
number of images:  23694


In [24]:
# # carpart_inference_results: a list of dictionaries: save inference results of carpart model on images 
# def inference_result(model, image):
    
#     result = inference_detector(model,image.copy())
#     out_image,pred_boxes,pred_segms,pred_labels,pred_scores = show_result_pyplot(model,image.copy(),result,score_thr=0.7)

#     carpart_inference_result = {}
#     carpart_inference_result[img_id] = {}
#     carpart_inference_result[img_id]['labels'] = pred_labels.tolist()
#     carpart_inference_result[img_id]['bboxes'] = (np.array(pred_boxes).astype(np.uint8)).tolist()
#     carpart_inference_result[img_id]['scores'] = pred_scores.tolist()
    
#     segs = []
#     for seg in pred_segms:
#         contours, hierarchy = cv2.findContours(np.array(seg).astype(np.uint8),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
#         for contour in contours:
#             contour = contour.tolist()
#             segs.append(contour)
#     carpart_inference_result[img_id]['segmentations'] = segs
    
#     return carpart_inference_result
    


In [ ]:
# carpart_inference_results = {}
# file_name = 'carpart_inference_results'
# json_path='../data/coco_datasets/datasets/scratch/annotations/'
# inference_results_path = os.path.join(json_path, file_name+'.json')

# with open(inference_results_path, 'w', encoding='utf-8') as json_file:
#     json.dump(carpart_inference_results, json_file)


# for img in tqdm(total_data['images']):
#     img_name = img['file_name']
#     img_id = img['id']
#     image = cv2.imread(os.path.join(imgs_folder, img_name))
    
#     carpart_inference_results.update(inference_result(carpart_model, image))

# with open(inference_results_path, mode='w', encoding='utf-8') as json_file:
#         json.dump(inference_results_path, json_file, ensure_ascii=False, indent=4)

 13%|█████████████████▋                                                                                                                 | 3191/23694 [24:42<58:23,  5.85it/s]

In [15]:
#load inference results
inference_results_path = '../data/coco_datasets/datasets/scratch/annotations/carpart_inference_results.json'

with open(inference_results_path, "r") as read_file:
    carpart_inference_results = json.load(read_file)
print(len(carpart_inference_results))

0


In [ ]:
# result0 = carpart_inference_results[0]
# print(result0)
# print(type(result0['bboxes']), result0['bboxes'].shape)
# print(type(result0['segmentations']), result0['segmentations'].shape)
# print(type(result0['labels']), result0['labels'].shape)

In [ ]:
#iterate through all images in image folder to get a dict of iou(imgId: iuos), ...

for img in tqdm(total_data['images']):
    img_id = img['id']
    img_name = img['file_name']
    image = cv2.imread(os.path.join(imgs_folder, img_name))
    anns = imgToAnns[img_id]
    
    #take predicted results of current img_id
    inference_results = carpart_inference_results[img_id]
    pred_boxes = inference_results['bboxes']
    pre_segms = inference_results['segmentations']
    pre_labels = inference_results['labels']
    
    if len(pred_segms) == 0:
        continue
    img_shape = pred_segms[0].shape
    
    #generate ground truth binary masks & compute iou
    for i in range(len(pred_labels)):
#         print('CARPART: {}'.format(classes[pred_labels[i]]), end = '  ')
        for j in range(len(anns)):
            ann = anns[j]
            ground_truth_seg = ann['segmentation'][0]
            
            damage_size = define_damage_size(ground_truth_seg,img_shape)
                
            ground_truth_mask = draw_binary_mask(ground_truth_seg, img_shape)

            predict_mask = draw_binary_mask(pred_segms[i], img_shape)  
            
            current_iou = iou(ground_truth_mask, predict_mask)
#             iou['img_id'].append(current_iou)
            if current_iou > 0:
                if damage_size == 'Small': small_scratch[classes[pred_labels[i]]] += 1
                elif damage_size == 'Medium': medium_scratch[classes[pred_labels[i]]] += 1
                else: large_scratch[classes[pred_labels[i]]] += 1
                
                scratches_on_carparts[classes[pred_labels[i]]] += 1

In [ ]:
#save dataframe
scratch_size = pd.DataFrame({'Carpart':small_scratch.keys(), 'Small damage':small_scratch.values(),
                             'Medium damage':medium_scratch.values(), 'Large damage':large_scratch.values()})
scratch_size.to_csv('statistical_charts/OLD_damaged_carpart_report.csv', index=False)
scratch_size

### Plot bar charts

In [ ]:
#plot number of scratches on each carparts
carparts = list(scratches_on_carparts.keys())
scratches = list(scratches_on_carparts.values())

fig = plt.figure(figsize=(10,5))
plt.barh(carparts,scratches)

plt.xlabel('number of scratches')
plt.ylabel('type of carparts')
plt.title('Scratches on carparts')
plt.savefig('statistical_charts/OLD_no_scratches.png')
plt.show()


In [ ]:
#plot number of large scratches on
carparts = list(large_scratch.keys())
no_large_scratch = list(large_scratch.values())

fig = plt.figure(figsize=(10,5))
plt.barh(carparts,no_large_scratch)

plt.xlabel('number of large scratches')
plt.ylabel('type of carparts')
plt.title('Large scratches on carparts')
plt.savefig('statistical_charts/OLD_no_large_scratches.png')
plt.show()


In [ ]:
#plot number of medium scratches on
carparts = list(medium_scratch.keys())
no_medium_scratch = list(medium_scratch.values())

fig = plt.figure(figsize=(10,5))
plt.barh(carparts,no_medium_scratch)

plt.xlabel('number of medium scratches')
plt.ylabel('type of carparts')
plt.title('Medium scratches on carparts')
plt.savefig('statistical_charts/OLD_no_medium_scratches.png')
plt.show()


In [ ]:
#plot number of small scratches on

carparts = list(small_scratch.keys())
no_small_scratch = list(small_scratch.values())

fig = plt.figure(figsize=(10,5))
plt.barh(carparts,no_small_scratch)

plt.xlabel('number of small scratches')
plt.ylabel('type of carparts')
plt.title('Small scratches on carparts')
plt.savefig('statistical_charts/OLD_no_small_scratches.png')
plt.show()


# Merimen data

In [ ]:
merimen_imgs_folder = '../data/coco_datasets/datasets/merimen_coco/19_02_2022/scratch/images/'
print(len([entry for entry in os.listdir(merimen_imgs_folder) if os.path.isfile(os.path.join(merimen_imgs_folder, entry))]))


In [12]:
#Load file total.json
merimen_json_path='../data/coco_datasets/datasets/merimen_coco/19_02_2022/scratch/annotations/total.json'
# with open(os.path.join(json_path, file_name+'.json'), 'w') as json_file:
#     json.dump(total_data, json_file, indent=4)
merimen_total_data = json.load(open(merimen_json_path))

In [13]:
#imgToAnns dict: mapping an image Id to a list of annotations
anns = {}
imgToAnns = defaultdict(list)
for ann in merimen_total_data['annotations']:
    anns[ann['id']] = ann
    imgToAnns[ann['image_id']].append(ann)


In [22]:
print('number of annotations: ', len(anns))
print('number of images: ', len(merimen_total_data['images']))
# print(imgId_to_img[2021].shape)

number of annotations:  120119
number of images:  23694


In [ ]:
merimen_carpart_inference_results = []
merimen_file_name = 'merimen_carpart_inference_results'
merimen_json_path='../data/coco_datasets/datasets/merimen_coco/19_02_2022/scratch/annotations/'
inference_results_path = os.path.join(merimen_json_path, merimen_file_name+'.json')

with open(inference_results_path, 'w', encoding='utf-8') as json_file:
    json.dump(merimen_carpart_inference_results, json_file)


for img in merimen_total_data['images']:
    img_name = img['file_name']
    image = cv2.imread(os.path.join(merimen_imgs_folder, img_name))
    
    with open(inference_results_path, "w",encoding='utf-8') as json_file:
        merimen_carpart_inference_results.append(inference_result(carpart_model, image))
        json.dump(merimen_carpart_inference_results, json_file)
print(len(merimen_carpart_inference_results))

In [ ]:
#load inference results
inference_results_path = '../data/coco_datasets/datasets/merimen_coco/19_02_2022/scratch/annotations/merimen_carpart_inference_results.json'

with open(inference_results_path, "r") as read_file:
    merimen_carpart_inference_results = json.load(read_file)
print(len(merimen_carpart_inference_results))

In [ ]:
result0 = merimen_carpart_inference_results[0]
print(result0)
print(type(result0['bboxes']), result0['bboxes'].shape)
print(type(result0['segmentations']), result0['segmentations'].shape)
print(type(result0['labels']), result0['labels'].shape)

In [ ]:
# iou_dict = defaultdict(list)
merimen_scratches_on_carparts = defaultdict(int)
merimen_small_scratch = defaultdict(int)
merimen_medium_scratch = defaultdict(int)
merimen_large_scratch = defaultdict(int)

In [ ]:
#iterate through all images in image folder to get a dict of iou(imgId: iuos), 
for i, (img_id, anns) in enumerate(imgToAnns.items()):
    print('imageid: {}'.format(img_id))
    
    #load image
    for img in merimen_total_data['images']:
        if img['id'] == img_id:
            img_name = img['file_name']
            image = cv2.imread(os.path.join(imgs_folder, img_name))
            break 
    
    #take predicted results
    inference_results = merimen_carpart_inference_results[i]
    pred_boxes = inference_results['bboxes']
    pre_segms = inference_results['segmentations']
    pre_labels = inference_results['labels']
    
    if len(pred_segms) == 0:
        continue
    img_shape = pred_segms[0].shape
    
    #generate ground truth binary masks & compute iou
    for i in range(len(pred_labels)):
#         print('CARPART: {}'.format(classes[pred_labels[i]]), end = '  ')
        for j in range(len(anns)):
            ann = anns[j]
            ground_truth_seg = ann['segmentation'][0]
            
            damage_size = define_damage_size(ground_truth_seg,img_shape)
                
            ground_truth_mask = draw_binary_mask(ground_truth_seg, img_shape)

            predict_mask = draw_binary_mask(pred_segms[i], img_shape)  
            
            current_iou = iou(ground_truth_mask, predict_mask)
#             iou['img_id'].append(current_iou)
            if current_iou > 0:
                if damage_size == 'Small': merimen_small_scratch[classes[pred_labels[i]]] += 1
                elif damage_size == 'Medium': merimen_medium_scratch[classes[pred_labels[i]]] += 1
                else: merimen_large_scratch[classes[pred_labels[i]]] += 1
                
                merimen_scratches_on_carparts[classes[pred_labels[i]]] += 1
#         print(scratches_on_carparts[classes[pred_labels[i]]])
                

In [ ]:
#save dataframe
merimen_scratch_size = pd.DataFrame({'Carpart':merimen_small_scratch.keys(), 'Small damage':merimen_small_scratch.values(),
                             'Medium damage':merimen_medium_scratch.values(), 'Large damage':merimen_large_scratch.values()})
merimen_scratch_size.to_csv('statistical_charts/MERIMEN_damaged_carpart_report.csv', index=False)
merimen_scratch_size

### Plot bar charts

In [ ]:
#plot number of scratches on each carparts
carparts = list(merimen_scratches_on_carparts.keys())
scratches = list(merimen_scratches_on_carparts.values())

fig = plt.figure(figsize=(10,5))
plt.barh(carparts,scratches)

plt.xlabel('number of scratches')
plt.ylabel('type of carparts')
plt.title('Scratches on carparts')
plt.savefig('statistical_charts/merimen_no_scratches.png')
plt.show()


In [ ]:
#plot number of large scratches on
carparts = list(merimen_large_scratch.keys())
no_large_scratch = list(merimen_large_scratch.values())

fig = plt.figure(figsize=(10,5))
plt.barh(carparts,no_large_scratch)

plt.xlabel('number of large scratches')
plt.ylabel('type of carparts')
plt.title('Large scratches on carparts')
plt.savefig('statistical_charts/merimen_no_large_scratches.png')
plt.show()


In [ ]:
#plot number of medium scratches on
carparts = list(merimen_medium_scratch.keys())
no_medium_scratch = list(merimen_medium_scratch.values())

fig = plt.figure(figsize=(10,5))
plt.barh(carparts,no_medium_scratch)

plt.xlabel('number of medium scratches')
plt.ylabel('type of carparts')
plt.title('Medium scratches on carparts')
plt.savefig('statistical_charts/merimen_no_medium_scratches.png')
plt.show()


In [ ]:
#plot number of small scratches on

carparts = list(merimen_small_scratch.keys())
no_small_scratch = list(merimen_small_scratch.values())

fig = plt.figure(figsize=(10,5))
plt.barh(carparts,no_small_scratch)

plt.xlabel('number of small scratches')
plt.ylabel('type of carparts')
plt.title('Small scratches on carparts')
plt.savefig('statistical_charts/merimen_no_small_scratches.png')
plt.show()
